In [1]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# Initialize the OpenAI client for Ollama
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# Step 1: Fetch and parse trending news from The Star (Kenya)
def fetch_trending_news():
    url = "https://thestar.co.ke/"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors

        soup = BeautifulSoup(response.text, 'html.parser')
        news_list = []

        # Look for headlines - SELECTORS MAY NEED ADJUSTMENT
        # Try to find common headline elements (h1, h2, h3, h4) with relevant classes
        headlines = soup.find_all(['h1', 'h2', 'h3', 'h4'], class_=lambda x: x != None)

        for headline in headlines[:10]:  # Get first 10 headlines
            headline_text = headline.get_text().strip()
            if headline_text and len(headline_text) > 20:  # Filter out short text
                news_list.append(headline_text)

        return news_list[:5]  # Return top 5 headlines
    except Exception as e:
        print(f"Error fetching news: {e}")
        return ["Failed to fetch trending news."]

# Step 2: Create your prompts using real news data
trending_news = fetch_trending_news()
news_text = "\n".join(trending_news)

system_prompt = "You are a news analyst specializing in Kenyan current affairs."
user_prompt = f"""
Based on the following trending news headlines from Kenya for today, provide a brief analysis of the main news topics:

{news_text}

Please identify 2-3 key themes and write a short summary (less than 300 words) about what's currently trending in Kenyan news.
"""

# Step 3: Make the messages list
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 4: Call Ollama
try:
    response = openai.chat.completions.create(model="llama3.2", messages=messages)
    # Step 5: Print the result
    print("=== TRENDING KENYAN NEWS ANALYSIS ===")
    print("\nToday's key headlines:")
    for i, headline in enumerate(trending_news, 1):
        print(f"{i}. {headline}")
    print("\n=== AI NEWS ANALYSIS ===")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error in AI analysis: {e}")

=== TRENDING KENYAN NEWS ANALYSIS ===

Today's key headlines:
1. Kenya Airways: 20 Facts About Africa’s Premier Airline You Need to Know
2. Sakaja Raises Nairobi Land Rates Effective January 2026
3. Motorists to Pay Ksh8 Per Kilometre to Use Rironi–Mau Summit Expressway – KeNHA
4. JSC Shortlists 100 Candidates for High Court Judge Positions
5. How to Check Nairobi Water Bill Online and Via SMS

=== AI NEWS ANALYSIS ===
After analyzing the trending news headlines from Kenya, I have identified three key themes:

1. **Infrastructure Development**: The first two headlines pertain to infrastructure development projects in Kenya. The article on Kenya Airways highlights the airline's importance in connecting Africa, while the story about Nairobi land rates raises questions about affordability and impact on developers. Meanwhile, motorists are set to face a new fee structure when using the Rironi-Mau Summit Expressway.
2. **E-Governance**: Two headlines touch upon e-governance initiatives. The